In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: cannot import name multiarray


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
inspector = inspect(engine)
session = Session(engine)

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
engine.execute('SELECT * FROM measurement').fetchall()

In [ ]:
all_data = engine.execute('SELECT * FROM station').fetchall()
all_data

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

last_12 = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= '2016-08-24').filter(Measurement.date <= '2017-08-23').order_by(Measurement.date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
last_12_data = pd.DataFrame(last_12)
last_12_data.set_index('date')
clean_last_12 = last_12_data.groupby('date').sum()
clean_last_12.head()

In [ ]:
clean_last_12 = last_12_data.dropna(axis=0, how='any')
precip = clean_last_12['prcp'].max()
print(f"The average precipitation is: {precip} inches")

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
# Define labels
plt.title("Precipitation for last 12 Months")
plt.xlabel("8/24/16 through 8/23/17")
plt.ylabel("Precipitation in inches")
plt.grid(False)


# Define X and Y values
y = clean_last_12["prcp"].tolist()
x = np.arange(0, len(clean_last_12.index.tolist()), 1)

#Plot

plot = plt.bar(x, y, width=25, color="#2d6bce", alpha=.7, align="edge")
plt.tick_params(labelbottom=False)


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
stats = clean_last_12.describe()
print(stats)

In [ ]:
# Design a query to show how many stations are available in this dataset?
all_stations = session.query(func.count(Station.station)).first()
all_stations

In [ ]:
# List the stations and the counts in descending order.
data = engine.execute('SELECT * FROM measurement').fetchall()
data

In [ ]:
data_df = pd.DataFrame(data)
clean_data_df = data_df.dropna(axis=0, how='any')

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?

most_active = session.query(Measurement.station, func.count(Measurement.station)).\
        group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
active_d = pd.DataFrame(most_active, columns=['Station', 'Count'])
most_active = active_d['Station'][0]
count = active_d['Count'][0]

In [ ]:
print(f"The most active station is: {most_active}")
print(f"Number of Observations: {count}")

In [ ]:
last_12_temp = session.query(Measurement.station, Measurement.tobs).\
        filter(Measurement.station == (most_active)).order_by(Measurement.date).all()
temp_df = pd.DataFrame(last_12_temp)
temp_df.head(3)
temp_df1 = temp_df.rename(index=str, columns={"tobs": "temps"})
temp_df1.head()

In [ ]:
#Set temps to print
high = int(temp_df1['temps'].max())
low = int(temp_df1['temps'].min())
mean = int(temp_df1['temps'].mean())

print(f"Temps at Station: USC00519281")
print(f"Date Range: 8/24/16 through 8/23/17")
print(f"High Temp: {high}")
print(f"Low Temp: {low}")
print(f"Avg Temp: {mean}")


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
grouped = temp_df1.groupby('temps').count().sort_values('station', ascending=False)
grouped.head()
temp_df1.hist('temps', bins=12)

###Frequencies is for ALL dates and not just 12 months - need to determine issue